In [87]:
# Import required libraries
import nltk
import numpy as np
import random
import json
import pickle
import os

# Set the custom NLTK data path
nltk.data.path.append('/content/nltk_data')

# Download only if necessary
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt', download_dir='/content/nltk_data')

nltk.download('punkt_tab', download_dir='/content/nltk_data')
nltk.download('wordnet', download_dir='/content/nltk_data')
nltk.download('stopwords', download_dir='/content/nltk_data')

from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# Load intents file
with open('dataset.json') as file:
    intents = json.load(file)

words = []
classes = []
documents = []
ignore_words = ['?']

# Tokenize and stem patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

# Training data
training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    pattern_words = [stemmer.stem(word.lower()) for word in doc[0]]
    for w in words:
        bag.append(1 if w in pattern_words else 0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training, dtype=object)
train_x = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

# Save words and classes
pickle.dump({'words': words, 'classes': classes, 'train_x': train_x, 'train_y': train_y}, open("training_data.pkl", "wb"))

# Build the model using Keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Build improved model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile with better optimizer
optimizer = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Train the model
model.fit(train_x, train_y, epochs=230, batch_size=6, verbose=1)

# Save the trained model
model.save('chatbot_model.keras')

[nltk_data] Downloading package punkt_tab to /content/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /content/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /content/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Epoch 1/230
726/726 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.0139 - loss: 7.2187
Epoch 2/230
726/726 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.0738 - loss: 6.2223
Epoch 3/230
726/726 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.1564 - loss: 5.4491
Epoch 4/230
726/726 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.2416 - loss: 4.6548
Epoch 5/230
726/726 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.2885 - loss: 4.0801
Epoch 6/230
726/726 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3287 - loss: 3.5150
Epoch 7/230
726/726 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.3781 - loss: 3.0823
Epoch 8/230
726/726 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.4295 - loss: 2.6689
Epoch 9/230
726/726 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.4696 - loss: 2.3079
Epoch 10/230
726/726 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5024 - loss: 2.1416
Epoch 11/230
726/726 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5261 - loss: 1.9884
Epoch 12/230
726/726 ━━━━━━━━━━━━━━━━━━━━

In [88]:
from keras.models import load_model

# Load model and data
model = load_model('chatbot_model.keras')
data = pickle.load(open("training_data.pkl", "rb"))
words = data['words']
classes = data['classes']

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print(f"found in bag: {w}")
    return np.array(bag)

# Classify input sentence
ERROR_THRESHOLD = 0.30

def classify(sentence):
    results = model.predict(np.array([bow(sentence, words)]), verbose=0)[0]
    results = [[i, r] for i, r in enumerate(results) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return [(classes[r[0]], r[1]) for r in results]

# Generate a response
def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    if results:
        for i in intents['intents']:
            if i['tag'] == results[0][0]:
                return random.choice(i['responses'])
    return "I don't understand that."

# Example usage:
print(response("Hello"))


Hi there. How are you feeling today?


In [89]:
print(response("i am feeling depressed"))

For a diagnosis of depression, a person needs to have experienced low mood or loss of interest or pleasure in life for at least 2 weeks. Also, they will have experienced the following symptoms: feelings of sadness, hopelessness, or irritability nearly every day.


In [91]:
print(response("i wanted to kill myself"))

helpline is available,you can contact the counsellor today:Befrienders Kenya helpline number is +254 722 178 177..always remember you mean something to someone.


In [92]:
print(response("i am useless.."))

For a diagnosis of depression, a person needs to have experienced low mood or loss of interest or pleasure in life for at least 2 weeks. Also, they will have experienced the following symptoms: feelings of sadness, hopelessness, or irritability nearly every day.


In [93]:
print(response("people makes fun of my disabilities"))

Relationships can be challenging for many reasons, but real connections are built on mutual respect and care. You are worthy of love, exactly as you are.


In [94]:
print(response("you are useless"))

I'm sorry to hear that. I'm doing my best to help


In [97]:
print(response("what is depression"))

Depression is a common and serious medical illness that negatively affects how you feel, the way you think and how you act.fortunately,it is also treatable.depression causes feelings of sadness and/or a loss of interest in activities you once enjoyed. it can lead to a variety of emotional and physical problems and can decrease your ability to function at work and at home.


In [98]:
print(response("how to deal with it"))

Start with your name and something simple about yourself, like a hobby or where you're from. It's okay to keep it brief!


In [99]:
print(response("Bye"))

Bye! Come back again.


In [100]:
print(response("i dont know why, but i am not liking the world for a very long time"))

Consider setting a specific goal that reflects your commitment to having a fulfilling career. Even a small daily action can reinforce your values.


In [101]:
print(response("do you really thinks somebody likes me?"))

That's no problem. I can see why you'd be stressed out about that. I can suggest you some tips to alleviate this issue. Would you like to learn more about that?


In [102]:
print(response("why no one wants to talks to me"))

That’s a hurtful feeling. You deserve to be heard. Try expressing how you feel to someone you trust, or consider journaling to process your thoughts.


In [103]:
print(response("i am willing to do suicide"))

I understand how you feel. Don't put yourself down because of it.


In [104]:
print(response("what do you think about me?"))

Consider setting a specific goal that reflects your commitment to empathy. Even a small daily action can reinforce your values.


In [105]:
print(response("Is online therapy as effective as in-person therapy?"))

Hi Reese, studies show that online therapy can be just as effective as in-person sessions, especially for anxiety and depression. The key is ensuring a private, comfortable space and consistent internet access.


In [106]:
print(response("I'm unsure if I need therapy or just a support group. How do I decide?"))

Skyler, a support group can be great if you're looking to connect with others facing similar challenges, while therapy offers personalized, one-on-one guidance. If your concerns are deep-rooted or interfering with daily life, therapy might be the better starting point.
